In [11]:
#importer les packages
import requests
import pandas as pd
import numpy as np
import configparser
from tqdm.notebook import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
import itertools 
import re
import time
import json
import pyreadstat
import dask.dataframe as dd
import dask
import concurrent

In [4]:
DATA_PATH="C:/Users/haallat/Documents/SAS_to_python-Atlas"

In [3]:
url='https://docs.google.com/spreadsheet/ccc?key=11NFXSIg6gQMCsMa8zWQQyypvvYBEmfyJfF2yytXqgMk&output=xls'

In [35]:
def get_all_correctifs_from_google():
    CORRECTIFS_dict= dict()
    VARS = ['COM_U','COM_M', 'ETABLI', 'MINISTER', 'DC', 'NATION', 'ETABLI2', 'FORMAT',
    'CURSUS_LMD', 'DISCIPLI','RGP2', 'RGP3','O_DUTBUT','J_LMDDONT','GDDISCIPLI']
    df_c = pd.read_excel(url, sheet_name=VARS, dtype=str)
    for VAR in VARS:
        correctifs = df_c.get(VAR).to_dict(orient='records')
        for c in correctifs:
            for f in c:
                if c[f] != c[f]: # nan
                    c[f] = ''
                if 'annee' in f.lower() or 'rentree' in f.lower():
                    c[f] = str(c[f])
                if isinstance(c[f], str):
                    c[f] = c[f].replace('.0','').strip()
                elif isinstance(c[f], float) or isinstance(c[f], int):
                    c[f] = str(c[f]).replace('.0','').strip()
        CORRECTIFS_dict[f'{VAR}'] = correctifs
    json.dump(CORRECTIFS_dict, open(f'{DATA_PATH}/correctifs.json', 'w'))

In [31]:
get_all_correctifs_from_google()

In [4]:
def get_all_correctifs():
    try:
        correctifs = json.load(open(f'{DATA_PATH}/correctifs.json', 'r'))
    except:
        correctifs = {}
    for k in correctifs:
        for ix, e in enumerate(correctifs[k]):
            for f in e.copy():
                if f.upper() != f:
                    correctifs[k][ix][f.upper()] = correctifs[k][ix][f]
    return correctifs

In [5]:
CORRECTIFS_dict = get_all_correctifs()

In [6]:
def corrige_com_u(df):
    VAR = 'COM_U'
    print(VAR)
    for c in CORRECTIFS_dict[VAR]:
        if c['ANNEE']!= '' :
            if c['COMPOS']!= '' :
                if c['IN'] == "' '":
                    df.loc[((df[VAR] == '')|(df[VAR] is np.nan)) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                else:
                    df.loc[(df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            else:
                if c['IN'] == "' '":
                    df.loc[((df[VAR] == '')|(pd.isna(df[VAR])) |(df[VAR]==None)) & (df['FINECOLE']==c['FINECOLE']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                else:
                    df.loc[(df['FINECOLE']==c['FINECOLE']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
        else:
            if c['COMPOS']!= '' :
                df.loc[(df['COMPOS']==c['COMPOS']), VAR]=c['OUT']
            else:
                if c['IN'] == "' '":
                    df.loc[(df[VAR] == '') & (df['FINECOLE']==c['FINECOLE']) , VAR]=c['OUT']
                else :
                    if c['FINECOLE'] != '':
                        df.loc[(df[VAR] == c['IN']) & (df['FINECOLE']==c['FINECOLE']) , VAR]=c['OUT']
                    else:
                        df.loc[(df[VAR] == c['IN']) , VAR]=c['OUT']
    return df

In [7]:
def corrige_com_m(df):
    VAR = 'COM_M'
    print(VAR)
    for c in CORRECTIFS_dict[VAR]:
        if c['ANNEE']!= '' :
            if c['COMPOS']!= '' :
                if c['IN'] == "' '":
                    df.loc[((df[VAR] == '')|(pd.isna(df[VAR])) |(df[VAR]==None)) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                elif c['ETABLI']!= '' :
                    if c['FORMAT']!= '' :
                        df.loc[(df['FORMAT']==c['FORMAT'])& (df['ETABLI']==c['ETABLI']) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                    else:
                        df.loc[(df['ETABLI']==c['ETABLI']) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                elif c['DIPLOM']!= '' :
                    df.loc[(df['DIPLOM']==c['DIPLOM']) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                elif c['COM_U']!= '' :
                    df.loc[((df['COM_U'] == "' '")|(df['COM_U'] is np.nan)) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                else:
                    if c['OUT'] == 'com_u':
                        df.loc[(df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=df['COM_U']
                    else:
                        df.loc[(df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            elif c['ETABLI']!= '' :
                df.loc[(df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            else:
                df.loc[(df['FINECOLE']==c['FINECOLE']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
        else:
            if c['COMPOS']!= '' :
                df.loc[(df['COMPOS']==c['COMPOS']), VAR]=c['OUT']
            else:
                if c['FINECOLE'] != '':
                    if c['IN'] == "' '":
                        df.loc[((df[VAR] == '')|(pd.isna(df[VAR])) |(df[VAR]==None)) & (df['FINECOLE']==c['FINECOLE']) , VAR]=c['OUT']
                    else:
                        df.loc[(df[VAR] == c['IN']) & (df['FINECOLE']==c['FINECOLE']) , VAR]=c['OUT']
                else :
                    df.loc[(df[VAR] == c['IN']) , VAR]=c['OUT']
    return df

In [8]:
def corrige_etabli(df):
    VAR = 'ETABLI'
    print(VAR)
    for c in CORRECTIFS_dict[VAR]:
        if c['ANNEE']!= '' :
            if c['COMPOS']!= '' :
                if c['IN'] != "''":
                    if c['FORMAT'] != '':
                        df.loc[(df[VAR] == c['IN']) & (df['FORMAT']==c['FORMAT']) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                    else:
                        df.loc[(df[VAR] == c['IN']) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                if c['DIPLOM'] != '':
                    df.loc[(df['DIPLOM']==c['DIPLOM']) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                else:
                    df.loc[(df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            else:
                if c['DIPLOM'] != '':
                    df.loc[(df[VAR] == c['IN']) & (df['DIPLOM']==c['DIPLOM']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                elif c['FORMAT'] != '':
                        df.loc[(df[VAR] == c['IN']) & (df['FORMAT']==c['FORMAT']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                elif c['CURSUS_LMD'] != '':
                        df.loc[(df[VAR] == c['IN']) & (df['CURSUS_LMD']==c['CURSUS_LMD']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
        else:
            df.loc[(df['COMPOS']==c['COMPOS']), VAR]=c['OUT']
    return df

In [9]:
def corrige_minister(df):
    VAR = 'MINISTER'
    print(VAR)
    for c in CORRECTIFS_dict[VAR]:
        if c['ANNEE']!= '' :
            df.loc[(df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
        else:
            df.loc[(df['ETABLI']==c['ETABLI']), VAR]=c['OUT']
    return df

In [10]:
def corrige_dc(df):
    VAR = 'DC'
    print(VAR)
    for c in CORRECTIFS_dict[VAR]:
        df.loc[(df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
    return df

In [11]:
def corrige_nation(df):
    VAR = 'NATION'
    print(VAR)
    for c in CORRECTIFS_dict[VAR]:
        df.loc[(df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
    return df

In [12]:
def corrige_etabli2(df):
    VAR = 'ETABLI2'
    print(VAR)
    for c in CORRECTIFS_dict[VAR]:
        if c['COMPOS']!= '' :
            df.loc[(df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
        else:
            if c['ACA_U']!= '' :
                df.loc[(df['ACA_U']==c['ACA_U']) & (df['FORMAT']==c['FORMAT']) & (df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            else:
                df.loc[(df['FORMAT']==c['FORMAT']) & (df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
    return df

In [13]:
def corrige_format(df):
    VAR = 'FORMAT'
    print(VAR)
    for c in CORRECTIFS_dict[VAR]:
        if c['ANNEE']!= '' :
            if c['SECT']!= '' :
                    df.loc[(df[VAR] == c['IN']) & (df['SECT']==c['SECT']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            elif c['COMPOS']!= '' :
                if c['IN']!= "''" :
                    df.loc[(df[VAR] == c['IN']) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                elif c['DIPLOM']!= '' :
                    df.loc[(df['DIPLOM'] == c['DIPLOM']) &(df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                else:
                    df.loc[(df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            else:
                if c['IN']!= "''" :
                    df.loc[(df[VAR] == c['IN']) & (df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                elif c['NOT IN']!= '' :
                    df.loc[(df[VAR] != c['NOT IN']) & (df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                elif c['TYP_DIPL']!= '' :
                    if c['NOT DIPLOM']!= '' :
                        df.loc[(df['DIPLOM'] != c['NOT DIPLOM']) & (df['TYP_DIPL']==c['TYP_DIPL']) & (df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                    else:
                        df.loc[(df['TYP_DIPL']==c['TYP_DIPL']) & (df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                else:
                    df.loc[(df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
        else:
            if c['IN']!= "''" :
                df.loc[(df[VAR] == c['IN']) & (df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            else:
                df.loc[(df[VAR] != c['NOT IN']) & (df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
    return df

In [14]:
def corrige_cursus_lmd(df):
    VAR = 'CURSUS_LMD'
    print(VAR)
    for c in CORRECTIFS_dict[VAR]:
        if c['ANNEE']!= '' :
            if c['ETABLI']!= '' :
                if c['IN']== "' '" :
                    df.loc[((df[VAR] == '')|(pd.isna(df[VAR])) |(df[VAR]==None)) & (df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                else:
                    df.loc[(df[VAR] == c['IN']) & (df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            elif c['TYP_DIPL']!= '' :
                df.loc[((df[VAR] == '')|(df[VAR] is np.nan)) & (df['TYP_DIPL']==c['TYP_DIPL']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            elif c['DIPLOM']!= '' :
                df.loc[((df[VAR] == '')|(df[VAR] is np.nan)) & (df['DIPLOM'] == c['DIPLOM']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            else:
                if c['IN']== "' '" :
                    df.loc[((df[VAR] == '')|(df[VAR] is np.nan)) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
                else:
                    df.loc[ (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
        else:
            df.loc[((df[VAR] == '')|(df[VAR] is np.nan)) & (df['ENQ']==c['ENQ']) & (df['ETABLI']==c['ETABLI']) , VAR]=c['OUT']
    return df

In [15]:
def corrige_discipli(df):
    VAR = 'DISCIPLI'
    print(VAR)
    for c in CORRECTIFS_dict[VAR]:
        if c['ETABLI']!= '' :
            df.loc[(df[VAR] == c['IN']) & (df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
        else:
            if c['IN']== "' '" :
                df.loc[((df[VAR] == '')|(pd.isna(df[VAR])) |(df[VAR]==None)) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            elif c['IN']== '' :
                df.loc[ (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
            else:
                df.loc[(df[VAR] == c['IN']) & (df['COMPOS']==c['COMPOS']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
    return df

In [16]:
def corrige_rgp2(df):
    VAR = 'RGP2'
    print(VAR)
    for c in CORRECTIFS_dict[VAR]:
        df.loc[(df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
    return df

In [17]:
def corrige_rgp3(df):
    VAR = 'RGP3'
    print(VAR)
    for c in CORRECTIFS_dict[VAR]:
        df.loc[(df['ETABLI']==c['ETABLI']) & (df['ANNEE']==c['ANNEE']), VAR]=c['OUT']
    return df

In [18]:
def corrige_all(df):
    return corrige_discipli(corrige_cursus_lmd(corrige_format(corrige_nation(corrige_dc(corrige_minister(corrige_etabli(corrige_com_m(corrige_com_u(df)))))))))


Importer les formats

In [21]:
communes=pyreadstat.read_sas7bdat(f'{DATA_PATH}/formats/les_communes.sas7bdat', encoding='iso-8859-1')[0]

In [22]:
communes

,COM_CODE,COM_CODE1,COM_CODE2,COM_ID,COM_NOM_MAJ_COURT,COM_NOM_MAJ,COM_NOM,UU_CODE,UU_ID,UUCR_ID,...,REG_NOM,REGRGP_NOM,REG_CODE_OLD,REG_ID_OLD,REG_NOM_OLD,FD_ID,FR_ID,FE_ID,UU_ID_99,UU_ID_10
0,01001,01001,01001,C01001,L ABERGEMENT CLEMENCIAT,L'ABERGEMENT-CLEMENCIAT,L'Abergement-Clémenciat,,SO,CR01001,...,Auvergne-Rhône-Alpes,Province,82,R82,Rhône-Alpes,FD111,FR11,FE1,SO,SO
1,01002,01002,01002,C01002,L ABERGEMENT DE VAREY,L'ABERGEMENT-DE-VAREY,L'Abergement-de-Varey,,SO,CR01002,...,Auvergne-Rhône-Alpes,Province,82,R82,Rhône-Alpes,FD111,FR11,FE1,SO,SO
2,01003,01003,01003,C01003,AMAREINS,AMAREINS,Amareins,,SO,SO,...,Auvergne-Rhône-Alpes,Province,82,R82,Rhône-Alpes,FD111,FR11,FE1,SO,SO
3,01004,01004,01004,C01004,AMBERIEU EN BUGEY,AMBERIEU-EN-BUGEY,Ambérieu-en-Bugey,01303,UU01303,UU01303,...,Auvergne-Rhône-Alpes,Province,82,R82,Rhône-Alpes,FD111,FR11,FE1,UU01303,UU01302
4,01005,01005,01005,C01005,AMBERIEUX EN DOMBES,AMBERIEUX-EN-DOMBES,Ambérieux-en-Dombes,,SO,CR01005,...,Auvergne-Rhône-Alpes,Province,82,R82,Rhône-Alpes,FD111,FR11,FE1,SO,SO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38960,99112,99112,99112,C99112,BUDAPEST,BUDASPEST,Dubapest,,SO,HON99112,...,Étranger,Étranger,99,R99,Étranger,FD290,FR29,FE2,SO,SO
38961,99036,99036,99036,C99036,Barcelonne,Barcelonne,Barcelonne,,SO,ESP99036,...,Étranger,Étranger,99,R99,Étranger,FD290,FR29,FE2,SO,SO
38962,99038,99038,99038,C99038,Oxford,Oxford,Oxford,,SO,SO,...,Étranger,Étranger,99,R99,Étranger,FD290,FR29,FE2,SO,SO
38963,99039,99039,99039,C99039,Stellenbosch,Stellenbosch,Stellenbosch,,SO,SO,...,Étranger,Étranger,99,R99,Étranger,FD290,FR29,FE2,SO,SO


In [23]:
all_secret = []
for annee in range(2001,2022):
    df_secret, meta_secret = pyreadstat.read_sas7bdat(f'{DATA_PATH}/formats/secret{str(annee)[2:4]}.sas7bdat', encoding='iso-8859-1')
    all_secret.append(df_secret)

In [24]:
secrets=pd.concat(all_secret)

In [25]:
all_format = []
for annee in range(2001,2022):
    df_format, meta_format = pyreadstat.read_sas7bdat(f'{DATA_PATH}/formats/format{str(annee)[2:4]}.sas7bdat', encoding='iso-8859-1')
    all_format.append(df_format)

In [26]:
formats=pd.concat(all_format)

In [6]:
all_df = []
for annee in range(2001,2022):
    print(annee)
    df = pyreadstat.read_sas7bdat(f'{DATA_PATH}/original/transup{str(annee)[2:4]}.sas7bdat', encoding='iso-8859-1')[0]
    df.columns = [x.upper() for x in df.columns]
    df['ANNEE']=df.loc[:,'ANNEE'].apply( lambda x: str(annee) )
    #print('correction')
    #if 'CURSUS_LMD' not in df.columns:
    #    df['CURSUS_LMD']=df.apply(lambda x: '', axis=1)
    #df=corrige_all(df)
    #print(df)
    #df.to_csv(f"atlas{str(annee)[2:4]}.csv", sep=";", encoding="UTF-8")
    df.to_csv(f"atlas/atlas{str(annee)[2:4]}.txt", header=True, index=False, sep=';', mode='a')
    #all_df.append(df)
        

parallèliser les corrections sur df_atlas

In [ ]:
def subset_df(df):
    prct100 = int(round(len(df) * 10 / 1000, 0))
    dict_nb = {}
    deb = 0
    fin = prct100
    dict_nb["df1"] = df.iloc[deb:fin, :]
    deb = fin + 1
    centième = 100 * prct100
    reste = (len(df) - centième)
    fin_reste = len(df) + 1
    for i in range(2, 101):
        fin = (i * prct100 + 1)
        dict_nb["df" + str(i)] = df.iloc[deb:fin, :]
        if reste > 0:
            dict_nb["reste"] = df.iloc[fin: fin_reste, :]
        deb = fin

    return dict_nb


In [ ]:
dict_nb=subset_df(df_atlas)

In [ ]:
dict_nb1=[dict_nb['df'+ str(x)] for x in range(1,101) ]

In [ ]:
dict_nb1=[dict_nb['df1'], dict_nb['df2']]

In [ ]:
def res_futures(dict_nb):
    res = []
    jointure=pd.DataFrame()
    with concurrent.futures.ThreadPoolExecutor(max_workers=11, thread_name_prefix="thread") as executor:
        # Start the load operations and mark each future with its URL
        future_to_req = {executor.submit(corrige_all, df_atlas): df for df in dict_nb}
        for future in concurrent.futures.as_completed(future_to_req):
            req = future_to_req[future]
            try:
                data = future.result()
                res.append(data)
                jointure = pd.concat(res)
            except:
                print('error')
                pass

    return jointure

In [ ]:
res_futures(dict_nb1)

In [ ]:
dict_nb['df100']['ANNEE']

Le code de Yann a part les corrections

In [19]:
df=pd.read_csv(f"atlas/atlas21.txt", sep=';',low_memory=False)

In [12]:
pd.Series.unique(df['TYP_DIPL'])

array(['', 'FI', 'XA', 'DP', 'ZF', 'XB', 'NA', 'RD', '13', '12', 'XD',
       'CB', 'DR', '17', '11', '01', '06', '09', 'RA', '05', '08', 'RB',
       'RH', 'PA', 'MB', 'MA', 'JD', 'CV', 'IB', 'PG', 'PL', 'PE', 'KG',
       '03', 'YR', 'NG', 'NH', 'FJ', 'JF', 'PF', 'QB', '18', 'ND', '02',
       'NC', 'FN', 'YI', '07', 'XF', 'YB', 'VF', 'UE', 'AC', 'UF', '24',
       '14', 'RG', 'UX', 'NI', 'FE', 'ZG', 'FH', 'JC', 'MD', 'UH', 'UC',
       '60', 'UA', 'CD', 'PB', 'PD', 'PK', 'YQ', 'PM', 'RC', 'VE', 'ZD',
       'VD', 'YM', 'SA', 'SP', 'SC', 'UJ', 'TA', 'TN', 'TC', 'TB', 'TH',
       'TG', 'TD', 'TE', 'TF', 'MC', 'PC', 'GG', 'VA', '63', '36', '10',
       'VG', 'UY', 'HH', 'SD', 'QA', '23', '35', '33', '34', 'VC', '62',
       '16', 'XE', '61', 'UD', 'NE', 'PI', 'YN', 'WB', 'NB', 'PH', 'NF',
       '32', 'UT', 'UP', 'UI', 'US', 'UU', '19', 'TP', 'TL', 'UW', 'UQ',
       'GF', 'FM', 'RE', '15'], dtype=object)

In [15]:
pd.Series.unique(df['DIPLOM'])

array(['', '6000336', '2300002', ..., '1000435', '2500336', '9010035'],
      dtype=object)

In [14]:
df['DIPLOM'] = df['DIPLOM'].fillna('0').astype(str)

In [35]:
df[-pd.isna(df['DIPLOM'])]['DIPLOM'][1546]

'6000336'

In [21]:
df['DISCIPLI']=df['DISCIPLI'].fillna('0').astype(str)

In [69]:
pd.DataFrame(CORRECTIFS_dict['O_DUTBUT'][0],index=[x for x in range(len(CORRECTIFS_dict['O_DUTBUT'][0]))])#[['DIPLOM','SPECIUT']]

,DIPLOM,correspondanceiut,SPECIUT,optiut,parcoursbut,LIB1,LIB2,CORRESPONDANCEIUT,OPTIUT,PARCOURSBUT
0,1000301,IUT_CJ,DUT_CJ,DUT_CJ_0,,CARRIERES JURIDIQUES,,IUT_CJ,DUT_CJ_0,
1,1000301,IUT_CJ,DUT_CJ,DUT_CJ_0,,CARRIERES JURIDIQUES,,IUT_CJ,DUT_CJ_0,
2,1000301,IUT_CJ,DUT_CJ,DUT_CJ_0,,CARRIERES JURIDIQUES,,IUT_CJ,DUT_CJ_0,
3,1000301,IUT_CJ,DUT_CJ,DUT_CJ_0,,CARRIERES JURIDIQUES,,IUT_CJ,DUT_CJ_0,
4,1000301,IUT_CJ,DUT_CJ,DUT_CJ_0,,CARRIERES JURIDIQUES,,IUT_CJ,DUT_CJ_0,
5,1000301,IUT_CJ,DUT_CJ,DUT_CJ_0,,CARRIERES JURIDIQUES,,IUT_CJ,DUT_CJ_0,
6,1000301,IUT_CJ,DUT_CJ,DUT_CJ_0,,CARRIERES JURIDIQUES,,IUT_CJ,DUT_CJ_0,
7,1000301,IUT_CJ,DUT_CJ,DUT_CJ_0,,CARRIERES JURIDIQUES,,IUT_CJ,DUT_CJ_0,
8,1000301,IUT_CJ,DUT_CJ,DUT_CJ_0,,CARRIERES JURIDIQUES,,IUT_CJ,DUT_CJ_0,
9,1000301,IUT_CJ,DUT_CJ,DUT_CJ_0,,CARRIERES JURIDIQUES,,IUT_CJ,DUT_CJ_0,


In [61]:
pd.DataFrame(CORRECTIFS_dict['GDDISCIPLI'])[['GD_DISCISCIPLINE','Discipline']].drop_duplicates()['Discipline'][0]


'15'

In [25]:
df_up=importtab(df,CORRECTIFS_dict,corrige_etabli2,corrige_all)

COM_U
COM_M
ETABLI
MINISTER
DC
NATION
FORMAT
CURSUS_LMD
DISCIPLI
ETABLI2


In [30]:
df_up

,SECTEUR,FORMAT,ENQ,NATION,ETABLI,COMPOS,NATR,SEXE,GROUPE,TYP_DIPL,...,LMDDONTBIS,GD_DISCISCIPLINE,Discipline,LMDONT,LMDONTBIS,MINISTER,DC,MIN,ETABLI2,ETABLI3
0,1,stspr,bpba,100,0010001W,0010001W,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"put(etabli,fEtabMin.)",0010001W,0010001W
1,1,stspr,bpba,100,0010001W,0010001W,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"put(etabli,fEtabMin.)",0010001W,0010001W
2,1,stspr,bpba,100,0010001W,0010001W,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"put(etabli,fEtabMin.)",0010001W,0010001W
3,1,stspr,bpba,100,0010001W,0010001W,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"put(etabli,fEtabMin.)",0010001W,0010001W
4,1,stspr,bpba,100,0010001W,0010001W,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"put(etabli,fEtabMin.)",0010001W,0010001W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2157054,3,soc,soc,100,9999999X,9999999X,1,.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"put(etabli,fEtabMin.)",9999999X,9999999X
2157055,3,soc,soc,100,9999999X,9999999X,1,.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"put(etabli,fEtabMin.)",9999999X,9999999X
2157056,3,soc,soc,100,9999999X,9999999X,1,.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"put(etabli,fEtabMin.)",9999999X,9999999X
2157057,3,soc,soc,100,9999999X,9999999X,1,.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"put(etabli,fEtabMin.)",9999999X,9999999X


In [ ]:
#fonction importtab

In [16]:
def importtab(df,CORRECTIFS_dict,corrige_etabli2,corrige_all):
    annee=df['ANNEE'][0]
    if annee in range(2001,2007) :
        df['LMDdont']=df.apply(lambda x: np.nan,axis=1)
        df['LMDdontbis']=df.apply(lambda x: np.nan,axis=1)
        df['DNDU']=df.apply(lambda x: np.nan,axis=1)
        df['DISCIPLI']=df.apply(lambda x: np.nan,axis=1)
        df['CURSUS_LMD']=df.apply(lambda x: np.nan,axis=1)
    elif annee in range(2001,2012) :
        df['FILIERE']=df.apply(lambda x: np.nan,axis=1)
        df['ETABLISSEMENT']=df.apply(lambda x: np.nan,axis=1)
    elif annee in range(2001,2015):
        df['UNIV']=df.apply(lambda x: np.nan,axis=1)
        df['HSIFA']=df.apply(lambda x: np.nan,axis=1)
    elif annee in range(2001,2018):
        df['EFFECTIF_SANS_DOUBLE_COMPTE']=df.apply(lambda x: np.nan,axis=1)
        df['HCPGE']=df.apply(lambda x: np.nan,axis=1) 
    
    if annee in range(2015,2022):
        df['DC']=df.apply(lambda x: 1,axis=1)
    df['DIPLOM'] = df['DIPLOM'].fillna(0)
    dutbut=pd.DataFrame(CORRECTIFS_dict['O_DUTBUT'][0],index=[x for x in range(len(CORRECTIFS_dict['O_DUTBUT'][0]))])
    dutbut['DIPLOM'] = dutbut['DIPLOM'].fillna('0').astype(str)
    df = pd.merge(df,dutbut[['DIPLOM','SPECIUT']],how= 'left', on='DIPLOM') 
    if annee in range(2007,2022):
        df.loc[(df['ETABLI']=='0381912X') & (df['TYP_DIPL'].isin(['FI','FN','YI'])) & (df['DIPLOM'].isin(['6001000','6004000','8000010'])),'FORMAT']='gding'
        df.loc[(df['TYP_DIPL'].isin(["01","02","04","05","06","07","08","09","17","UA","UB","UC","UD","UE","UF","UG","UI","UJ","UO","UR","US","UT","UU","UV","UY","XF"])),'DNDU']='DU'
        df.loc[df['TYP_DIPL'].isin(["01","02","04","05","06","07","08","09","17","UA","UB","UC","UD","UE","UF","UG","UI","UJ","UO","UR","US","UT","UU","UV","UY","XF"]),'DNDU']='DN'
        df = pd.merge(df,pd.DataFrame(CORRECTIFS_dict['J_LMDDONT'][0],index=[x for x in range(len(CORRECTIFS_dict['J_LMDDONT'][0]))]),how= 'left', on='TYP_DIPL') 
        df.loc[df['DNDU']=='DU','LMDdontbis']='DU'
        dg_disc=pd.DataFrame(CORRECTIFS_dict['GDDISCIPLI'])[['GD_DISCISCIPLINE','Discipline']].drop_duplicates()
        df = pd.merge(df,dg_disc,how= 'left', left_on='DISCIPLI',right_on='Discipline') 
        df.loc[df['DIPLOM'].isin(['6001000','6004000','8000010']),'LMDONT']='AUTRES'
        df.loc[df['DIPLOM'].isin(['6001000','6004000','8000010']),'LMDONTBIS']='AUTRES'   
    df=corrige_all(df)
    if annee==2016:
        df.loc[(df['COMPOS']=='0684045X'),'COMPOS']='0694045X'
        df.loc[(df['COMPOS']=='0684045X'),'MIN_M']='0694045X'
        df.loc[(df['COMPOS']=='0684045X'),'MIN_U']='0694045X'
        df.loc[(df['COMPOS']=='0684045X'),'NAT_M']='0694045X'
        df.loc[(df['COMPOS']=='0684045X'),'NAT_U']='0694045X'
    elif annee==2015:   
        df.loc[(df['COMPOS']=='0755359T')& (pd.isna(df['MIN_M'])),'MIN_U']='38'
        df.loc[(df['COMPOS']=='0755359T')& (pd.isna(df['MIN_M'])),'MIN_M']='38'
        df.loc[(df['COMPOS']=='0161192P')& (pd.isna(df['MIN_M'])),'MIN_U']='38'
        df.loc[(df['COMPOS']=='0161192P')& (pd.isna(df['MIN_M'])),'MIN_U']='38'
    elif annee==2015:    
        df.loc[(df['COMPOS']=='0410981U'),'COM_M']='18033'
        df.loc[(df['COMPOS']=='0410981U'),'DEP_ID_etab']='D018'
    elif annee==2013:
        df.loc[((df['ETABLI'] == "0692459Y")|(df['ETABLI'] == "0753478Y")|(df['ETABLI'] == "0753486G")|(df['ETABLI'] == "0753494R")|(df['ETABLI'] == "0753742K")|(df['ETABLI'] == "0133968T")|(df['ETABLI'] == "0782019W")),'SECT']='PU'
        df.loc[(df['COMPOS']=='0490890B'),'SECT']='PU'
        df.loc[(df['FORMAT']=='CPESn') & (df['CPESN'] > 0) & (df['CPESN'] != df['EFFTOTN']),'EFFTOTN']=df['CPESN']
    elif annee==2012:
        df.loc[(df['COMPOS']=='0490890B'),'SECT']='PU'
        df.loc[(df['FORMAT']=='CPESn') & (df['CPESN'] > 0) & (df['CPESN'] != df['EFFTOTN']),'EFFTOTN']=df['CPESN']
    df.loc[df['COMPOS']=='0673064S','SECT']='PR'
    df['MIN']=df.loc[:,'MINISTER'].apply(lambda x: x if ((x == '')|(x == None)|(pd.isna(x)==False)) else "put(etabli,fEtabMin.)")
    df.loc[df['ETABLI']=='0772517T','MIN']='23'
    df.loc[df['ETABLI']=='0772517T','MINISTER']='23'
    df['ETABLI2']=df.loc[:,'ETABLI']
    df=corrige_etabli2(df)
    df['ETABLI3']=df.loc[:,'ETABLI']
    df.loc[(df['ETABLI']=='0753541S') & pd.isna(df['SECT']),'SECT']='PU'
    final_columns=['ANNEE','RENTREE','ENQ','MIN','MINISTER','SECT','COMPOS','NAT_U','SIGLE_U','LIB1_U','LIB2_U','COM_U','FINECOLE','ETABLI','ETABLI2','ETABLI3','SIGLE_M','LIB1_M','LIB2_M','COM_M','SPECIUT','EFFTOT','EFFTOTN','DC','LMDdont','LMDdontbis','DNDU','DISCIPLI','CURSUS_LMD','FILIERE','ETABLISSEMENT','UNIV','HSIFA','EFFECTIF_SANS_DOUBLE_COMPTE','HCPGE','SEXE','NATION']
    columns=[ x for x in df.columns if x in final_columns]
    df[columns]
    df.to_csv(f"atlas_up/atlas{str(annee)[2:4]}.txt", header=True, index=False, sep=';', mode='a')
    return df

In [ ]:
#on execute la fonction importtab pour chaque année

In [ ]:
['RENTREE', 'ENQ', 'MIN', 'MINISTER', 'SECT', 'COMPOS', 'NAT_U', 'SIGLE_U', 'LIB1_U',
    'LIB2_U', 'COM_CODE', 'COM_NOM', 'COM_CODE1', 'COM_NOM1', 'COM_CODE2', 'COM_NOM2', 'UUCR_ID', 
    'UUCR_NOM', 'SACLAY_ID', 'EPCI_ID', 'DEP_ID', 'DEP_NUM_NOM', 'ACA_ID', 'ACA_NOM', 'REG_ID', 'REG_NOM',
    'REG_ID_OLD', 'REG_NOM_OLD', 'REGRGP_NOM', 'FINECOLE', 'ETABLI', 'SECRET', 'SIGLE_M', 'LIB1_M', 'LIB2_M',
    'COM_CODE_ETAB', 'UUCR_ID_ETAB', 'DEP_ID_ETAB', 'ETABLI2', 'ETABLI3', 'FORMAT', 'SPECIUT',  'LIBELL_', 
    'RGP', 'RGP2', 'RGP3', 'RGP4', 'ING', 'IUT', 'INSPE', 'TOTAL', 'MEMEUUCR', 'MEMECOM']

In [ ]:
['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','rgp3']

In [ ]:
#fonction gentab

In [ ]:
def gentab(df):
    annee=df['ANNEE'][0]
    df['EFFSDC']=df.loc[:,'HCPGE']
    df['EFFSIFA']=df[:,'HSIFA']
    df1=df.loc[df['DC']== 1 & df['EFFTOT']>0,:]
    df2=df.loc[df['DC']!= 1 | df['EFFTOT']==0,:]
    array=['RENTREE', 'ENQ', 'MIN', 'MINISTER', 'SECT', 'COMPOS', 'NAT_U', 'SIGLE_U', 'LIB1_U',
    'LIB2_U','FINECOLE', 'ETABLI', 'SIGLE_M', 'LIB1_M', 'LIB2_M','COM_M', 'COM_M', 'ETABLI2', 
    'ETABLI3', 'FORMAT', 'SPECIUT']
    f = {'EFFTOT': 'sum', 'EFFSDC': 'sum', 'EFFSIFA': 'sum'}
    df1=df.groupby(array, as_index=False).agg(f) 
    df=pd.concat([df1,df2])
    communes=pyreadstat.read_sas7bdat(f'{DATA_PATH}/formats/les_communes.sas7bdat', encoding='iso-8859-1')[0]
    df = pd.merge(df,communes,how= 'left', left_on='COM_U',right_on='COM_CODE') 
    df = pd.merge(df,communes,how= 'left', left_on='COM_M',right_on='COM_CODE') 
    format=pyreadstat.read_sas7bdat(f'{DATA_PATH}/formats/format{str(annee)[2:4]}.sas7bdat', encoding='iso-8859-1')[0]
    format.columns = [x.upper() for x in format.columns]
    df = pd.merge(df,format,how= 'left', on='FORMAT') 
    secret=pyreadstat.read_sas7bdat(f'{DATA_PATH}/formats/secret{str(annee)[2:4]}.sas7bdat', encoding='iso-8859-1')[0]
    secret.columns = [x.upper() for x in format.columns]
    df = pd.merge(df,format,how= 'left', on='ETABLI') 
    df = pd.merge(df,communes,how= 'left', left_on='COM_CODE1',right_on='COM_CODE') 
    df = pd.merge(df,communes,how= 'left', left_on='COM_CODE2',right_on='COM_CODE') 
    df['ID']=df.loc[:,'REGION_ID']
    df['EFFECTIF_AVEC_SIFA']=df.loc[:,'EFFTOT']
    df['EFFTOT']=df.loc[:,'EFFTOT']-df['EFFSIFA']
    df['EFFSDC']=df.loc[:,'EFFSDC']-df['EFFSIFA']
    df.loc[((df['ING'] == '')|(pd.isna(df['ING'])) |(df['ING']==None)),'ING']='NO_ING'
    df.loc[((df['IUT'] == '')|(pd.isna(df['IUT'])) |(df['IUT']==None)),'IUT']='NO_IUT'
    df.loc[((df['INSPE'] == '')|(pd.isna(df['INSPE'])) |(df['INSPE']==None)),'INSPE']='NO_INSPE'
    df.loc[((df['INSPE'] != '')|(pd.isna(df['INSPE'])==False) |(df['INSPE']!=None)|(df['INSPE']!='NO_INSPE')),'INSPE']='NO_INSPE'
    df.loc[((df['IUT'] != '')|(pd.isna(df['IUT'])==False) |(df['IUT']!=None)|(df['IUT']!='IUT')) & (df['SPECIUT']=='AUTRES'),'IUT']='NO_IUT'
    df.loc[(df['REG_ID'] in ["R01","R02","R03","R04","R06"]),'ID']='FD112'
    del df['EFFSIFA']
    return df

    

In [ ]:
def gentab2(df):
    df.loc[df['ING']=='ING' & df['MIN']=='38' & df['SECT']=='PU' & df['RGP3'] == 'UNIV','RGP_ING']='ING_UNIV'
    df.loc[df['ING']=='ING' & df['MIN']=='38' & df['SECT']=='PU' & df['RGP3'] == 'GE','RGP_ING']='ING_GDETAB'
    df.loc[df['ING']=='ING' & df['MIN']=='38' & df['SECT']=='PU' & df['RGP3'] == 'GE','RGP_ING2']='ING_GD_INP_UT'
    df.loc[df['ING']=='ING' & df['MIN']=='38' & df['SECT']=='PU' & df['RGP3'] in ['INP','UT'],'RGP_ING']='ING_INP_UT'
    df.loc[df['ING']=='ING' & df['MIN']=='38' & df['SECT']=='PU' & df['RGP3'] in ['INP','UT'],'RGP_ING2']='ING_GD_INP_UT'
    df.loc[df['ING']=='ING' & df['MIN']=='38' & df['SECT']=='PU' & pd.isna(df['RGP_ING']),'RGP_ING']='ING_MESR_PU'
    df.loc[df['ING']=='ING' & df['MIN']=='38' & df['SECT']=='PU' & df['RGP3'] not in ['UNIV','GE','INP','UT'],'RGP_ING']='ING_PR'
    #min=38 mais pas public
    df.loc[df['ING']=='ING' & df['MIN']=='38' & df['SECT']!='PU','RGP_ING']='ING_PR'
    # pas min=38
    df.loc[df['ING']=='ING' & df['MIN']!='38' & df['SECT']=='PU','RGP_ING']='ING_AUT_MIN_PU'
    df.loc[df['ING']=='ING' & df['MIN']!='38' & df['SECT']!='PU','RGP_ING']='ING_PR'
    # pas inge
    df.loc[df['ING']!='ING','RGP_ING']='NO_ING'
    df.loc[pd.isna(df['RGP_ING2']),'RGP_ING']='NO_ING'

    df1=df.loc[df['RGP3'] not in ['CPGE','STS','UNIV','GE','INP','UT','ESPE'],:]
    array=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','RGP3']
    f = {'EFFTOT': 'sum'}
    df_COM_AUT=df1.groupby(array, as_index=False).agg(f)
    
    df1=df.loc[df['RGP3'] in ['CPGE','STS'],:]
    array=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','RGP3']
    f = {'EFFTOT': 'sum'}
    df_COM_CSI=df1.groupby(array, as_index=False).agg(f)

    df1=df.loc[df['RGP3'] in ['UNIV','GE','INP','UT'] & df['IUT']!='IUT' & df['INSPE']!='INSPE' & df['RGP2']!='EPE',:]
    array=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','RGP3','RGP2','COM_M']
    f = {'EFFTOT': 'sum'}
    df_COM_UNIV=df1.groupby(array, as_index=False).agg(f)

    df1=df.loc[df['IUT'] == 'IUT',:]
    array=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','RGP3','RGP2','COM_M']
    f = {'EFFTOT': 'sum'}
    df_COM_IUT=df1.groupby(array, as_index=False).agg(f)
    df_COM_IUT.loc[:,'RGP3']="IUT"

    df1=df.loc[df['INSPE'] == 'ESPE',:]
    array=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','INSPE','COM_M']
    f = {'EFFTOT': 'sum'}
    df_COM_ESPE=df1.groupby(array, as_index=False).agg(f)
    df_COM_ESPE.loc[:,'RGP3']="INSPE"

    df1=df.loc[df['RGP2'] == 'EPE',:]
    array=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','RGP2','COM_M']
    f = {'EFFTOT': 'sum'}
    df_COM_EPE=df1.groupby(array, as_index=False).agg(f)
    df_COM_EPE['RGP3']=df1.loc[:,'RGP2']

    df_COM_UNIV.loc[df['COM_U']!=1,'RGP3']=f"{df['RGP3']}_S"
    (df_COM_CSI.pivot_table(index=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1'], columns=['RGP3','EFFTOT'], values='spend')
              .add_prefix('spend_')
              .reset_index())
    
    df_COM_CSI.loc[df_COM_CSI['CPGE']!='.'&df_COM_CSI['STS']=='.','RGP3']='CPGE'
    df_COM_CSI.loc[df_COM_CSI['CPGE']!='.'&df_COM_CSI['STS']!='.','RGP3']='STS_CPGE'
    df_COM_CSI.loc[df_COM_CSI['CPGE']=='.'&df_COM_CSI['STS']!='.','RGP3']='STS'
    df_COM_CSI['EFFTOT']=sum(df_COM_CSI['CPGE'],df_COM_CSI['STS'])
    del df_COM_CSI['CPGE']
    del df_COM_CSI['STS']

    df_COM_INV=pd.concat([df_COM_UNIV,df_COM_CSI,df_COM_AUT,df_COM_IUT,df_COM_ESPE])
    df_COM_INV.loc[:,'UAI']=f"{df.loc[:,'COMECODE1']}_{df.loc[:,'RGP3']}"
    df_COM_INV.loc[:,'GEO_LIB']=df.loc[:,'COM_NOM1']
    df_COM_INV.loc[:,'GEO_ID']=df.loc[:,'COME_CODE1']
    df_COM_INV.loc[:,'REF_ID']=df.loc[:,'DEP_ID']
    df_COM_INV.loc[:,'TYPE2']=df.loc[:,'RGP3']
    del df_COM_INV['RENTREE']
    del df_COM_INV['DEP_ID']
    del df_COM_INV['COM_CODE1']
    del df_COM_INV['COM_NOM1']
    del df_COM_INV['RGP3']
    del df_COM_INV['RGP2']
    del df_COM_INV['INSPE']

    df_COM_SIEGE_UNIV=df.loc[df['COM_U']==1 & df['RGP3'] == 'UNIV',:]
    df_COM_SIEGE_UNIV.loc[:,'GEO_LIB']=df.loc[:,'COM_NOM1']
    df_COM_SIEGE_UNIV.loc[:,'GEO_ID']=df.loc[:,'COME_CODE1']
    df_COM_SIEGE_UNIV.loc[:,'REF_ID']=df.loc[:,'DEP_ID']

    array=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    f = {'EFFTOT': 'sum'}
    df_COM_SIEGE_UNIV=df_COM_SIEGE_UNIV.groupby(array, as_index=False).agg(f)

    df_COM_SIEGE=df.loc[df['COM_U']==1 & (df['RGP3'] in ['UNIV','GE','INP','UT','ENS','ESPE','IUFM']| df['RGP_ING']== 'ING_MESR_PU'),:]
    df_COM_SIEGE.loc[:,'GEO_LIB']=df.loc[:,'COM_NOM1']
    df_COM_SIEGE.loc[:,'GEO_ID']=df.loc[:,'COME_CODE1']
    df_COM_SIEGE.loc[:,'REF_ID']=df.loc[:,'DEP_ID']

    array=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    f = {'EFFTOT': 'sum'}
    df_COM_SIEGE=df_COM_SIEGE.groupby(array, as_index=False).agg(f)

    df_COM_UNIV=df.loc[df['RGP3'] =='UNIV',:]
    df_COM_UNIV.loc[:,'GEO_LIB']=df.loc[:,'COM_NOM1']
    df_COM_UNIV.loc[:,'GEO_ID']=df.loc[:,'COME_CODE1']
    df_COM_UNIV.loc[:,'REF_ID']=df.loc[:,'DEP_ID']

    array=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    f = {'EFFTOT': 'sum'}
    df_COM_UNIV=df_COM_UNIV.groupby(array, as_index=False).agg(f)

    df_COM_UNIVERSITAIRE=df.loc[df['RGP3'] in ['UNIV','GE','INP','UT','ENS','ESPE','IUFM']| df['RGP_ING']== 'ING_MESR_PU',:]
    df_COM_UNIVERSITAIRE.loc[:,'GEO_LIB']=df.loc[:,'COM_NOM1']
    df_COM_UNIVERSITAIRE.loc[:,'GEO_ID']=df.loc[:,'COME_CODE1']
    df_COM_UNIVERSITAIRE.loc[:,'REF_ID']=df.loc[:,'DEP_ID']

    array=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    f = {'EFFTOT': 'sum'}
    df_COM_UNIVERSITAIRE=df_COM_UNIVERSITAIRE.groupby(array, as_index=False).agg(f)
    
    df_COM_SITES=df_COM_INV.groupby(array, as_index=False).agg(f)

    col=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    df_COM_SITES.loc[df_COM_UNIVERSITAIRE[col].isin(df_COM_SITES[col]),'UNIVERSITAIRE']='ok'
    df_COM_SITES.loc[df_COM_SIEGE[col].isin(df_COM_SITES[col]),'SIEGE']='ok'
    df_COM_SITES.loc[df_COM_SIEGE_UNIV[col].isin(df_COM_SITES[col]),'SIEGE_UNIV']='ok'
    df_COM_SITES.loc[df_COM_UNIV[col].isin(df_COM_SITES[col]),'UNIV']='ok'
    df_COM_SITES.loc[df_COM_SIEGE[col].isin(df_COM_SITES[col]),'TYPE_SITE']='site_princ'
    df_COM_SITES.loc[df_COM_UNIVERSITAIRE[col].isin(df_COM_SITES[col]),'TYPE_SITE']='site_sec'
    df_COM_SITES.loc[(-df_COM_UNIVERSITAIRE[col].isin(df_COM_SITES[col]))|(-df_COM_SIEGE.isin(df_COM_SITES)[col]),'TYPE_SITE']='site_sec'
    
    df_UUCR_AUT=df.loc[df['RGP3'] not in ['CPGE','STS','UNIV','GE','INP','UT'],:]
    array=['RENTREE','REG_ID','UUCR_ID','UUCR_NOM','RGP3']
    f = {'EFFTOT': 'sum'}
    df_UUCR_AUT=df_UUCR_AUT.groupby(array, as_index=False).agg(f)
     
    df_UUCR_CSI=df.loc[df['RGP3'] in ['CPGE','STS'],:]
    array=['RENTREE','REG_ID','UUCR_ID','UUCR_NOM','RGP3']
    f = {'EFFTOT': 'sum'}
    df_UUCR_CSI=df_UUCR_CSI.groupby(array, as_index=False).agg(f)
    
    df_UUCR_UNIV=df.loc[df['RGP3'] in ['UNIV','GE','INP','UT'] & df['IUT'] !='IUT' & df['INSPE'] !='ESPE' & df['RGP2'] !='EPE' ,:]
    array=['RENTREE','REG_ID','UUCR_ID','UUCR_NOM','RGP3','UUCR_ID ou memeUUCR']
    f = {'EFFTOT': 'sum'}
    df_UUCR_UNIV=df_UUCR_UNIV.groupby(array, as_index=False).agg(f)
    
    df_UUCR_IUT=df.loc[ df['IUT'] =='IUT',:]
    array=['RENTREE','REG_ID','UUCR_ID','UUCR_NOM','IUT','UUCR_ID ou memeUUCR']
    f = {'EFFTOT': 'sum'}
    df_UUCR_IUT=df_UUCR_IUT.groupby(array, as_index=False).agg(f)
    df_UUCR_IUT.loc[:,'RGP3']=df_UUCR_IUT.loc[:,'IUT']
    
    df_UUCR_ESPE=df.loc[ df['INSPE'] =='ESPE',:]
    array=['RENTREE','REG_ID','UUCR_ID','UUCR_NOM','INSPE','UUCR_ID ou memeUUCR']
    f = {'EFFTOT': 'sum'}
    df_UUCR_ESPE=df_UUCR_ESPE.groupby(array, as_index=False).agg(f)
    
    df_UUCR_EPE=df.loc[ df['RGP2'] =='EPE',:]
    array=['RENTREE','REG_ID','UUCR_ID','UUCR_NOM','RGP2','UUCR_ID ou memeUUCR']
    f = {'EFFTOT': 'sum'}
    df_UUCR_EPE=df_UUCR_EPE.groupby(array, as_index=False).agg(f)
    df_UUCR_EPE.loc[:,'RGP3']=df_UUCR_IUT.loc[:,'RGP2']
    
    df_UUCR_UNIV.loc[df_UUCR_UNIV['memeUUCR']!=1,'RGP3']=f"{df_UUCR_UNIV.loc[:,'RGP3']}_S"
    (df_UUCR_CSI.pivot_table(index=['RENTREE','REG_ID','UUCR_ID','UUCR_NOM'], columns=['RGP3','EFFTOT'], values='spend')
              .add_prefix('spend_')
              .reset_index())
    
    df_UUCR_CSI.loc[df_UUCR_CSI['CPGE']!='.'&df_UUCR_CSI['STS']=='.','RGP3']='CPGE'
    df_UUCR_CSI.loc[df_UUCR_CSI['CPGE']!='.'&df_UUCR_CSI['STS']!='.','RGP3']='STS_CPGE'
    df_UUCR_CSI.loc[df_UUCR_CSI['CPGE']=='.'&df_UUCR_CSI['STS']!='.','RGP3']='STS'
    df_UUCR_CSI['EFFTOT']=sum(df_UUCR_CSI['CPGE'],df_UUCR_CSI['STS'])
    del df_UUCR_CSI['CPGE']
    del df_UUCR_CSI['STS']
    
    df_UUCR_INV=pd.concat([df_COM_UNIV,df_COM_CSI,df_COM_AUT,df_COM_IUT,df_COM_ESPE])
    df_UUCR_INV.loc[:,'UAI']=f"{df.loc[:,'COMECODE1']}_{df.loc[:,'RGP3']}"
    df_UUCR_INV.loc[:,'GEO_LIB']=df.loc[:,'COM_NOM1']
    df_UUCR_INV.loc[:,'GEO_ID']=df.loc[:,'COME_CODE1']
    df_UUCR_INV.loc[:,'REF_ID']=df.loc[:,'DEP_ID']
    df_UUCR_INV.loc[:,'TYPE2']=df.loc[:,'RGP3']
    del df_UUCR_INV['RENTREE']
    del df_UUCR_INV['DEP_ID']
    del df_UUCR_INV['COM_CODE1']
    del df_UUCR_INV['COM_NOM1']
    del df_UUCR_INV['RGP3']
    del df_UUCR_INV['RGP2']
    del df_UUCR_INV['INSPE']
    
    df_UUCR_SIEGE=df.loc[df['memeUUCR']==1 &(df['RGP3'] == ['UNIV','GE','INP','UT','ENS','ESPE','IUFM']|df['RGP_ING'] == 'ING_MESR_PU'),:]
    df_UUCR_SIEGE.loc[:,'GEO_LIB']=df_UUCR_SIEGE.loc[:,'COM_NOM1']
    df_UUCR_SIEGE.loc[:,'GEO_ID']=df_UUCR_SIEGE.loc[:,'COME_CODE1']
    df_UUCR_SIEGE.loc[:,'REF_ID']=df_UUCR_SIEGE.loc[:,'DEP_ID']
    
    array=['REF_ID','GEO_ID','GEO_LIB']
    f = {'EFFTOT': 'sum'}
    df_UUCR_UNIV=df_UUCR_UNIV.groupby(array, as_index=False).agg(f)

    df_UUCR_SIEGE_UNIV=df.loc[df['COM_U']==1 & df['RGP3'] == 'UNIV',:]
    df_UUCR_SIEGE_UNIV.loc[:,'GEO_LIB']=df_UUCR_SIEGE_UNIV.loc[:,'COM_NOM1']
    df_UUCR_SIEGE_UNIV.loc[:,'GEO_ID']=df_UUCR_SIEGE_UNIV.loc[:,'COME_CODE1']
    df_UUCR_SIEGE_UNIV.loc[:,'REF_ID']=df_UUCR_SIEGE_UNIV.loc[:,'DEP_ID']
    
    array=['REF_ID','GEO_ID','GEO_LIB']
    f = {'EFFTOT': 'sum'}
    df_UUCR_UNIV=df_UUCR_UNIV.groupby(array, as_index=False).agg(f)
    
    df_UUCR_UNIVERSITAIRE=df.loc[df['RGP3'] in ['UNIV','GE','INP','UT','ENS','ESPE','IUFM']| df['RGP_ING']== 'ING_MESR_PU',:]
    df_UUCR_UNIVERSITAIRE.loc[:,'GEO_LIB']=df.loc[:,'COM_NOM1']
    df_UUCR_UNIVERSITAIRE.loc[:,'GEO_ID']=df.loc[:,'COME_CODE1']
    df_UUCR_UNIVERSITAIRE.loc[:,'REF_ID']=df.loc[:,'DEP_ID']

    array=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    f = {'EFFTOT': 'sum'}
    df_UUCR_UNIVERSITAIRE=df_UUCR_UNIVERSITAIRE.groupby(array, as_index=False).agg(f)

    array=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    f = {'EFFTOT': 'sum'}
    df_UUCR_SITES=df_UUCR_INV.groupby(array, as_index=False).agg(f)

    col=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    df_UUCR_SITES.loc[df_UUCR_UNIVERSITAIRE[col].isin(df_UUCR_SITES[col]),'UNIVERSITAIRE']='ok'
    df_UUCR_SITES.loc[df_UUCR_SIEGE[col].isin(df_UUCR_SITES[col]),'SIEGE']='ok'
    df_UUCR_SITES.loc[df_UUCR_SIEGE_UNIV[col].isin(df_UUCR_SITES[col]),'SIEGE_UNIV']='ok'
    df_UUCR_SITES.loc[df_UUCR_UNIV[col].isin(df_UUCR_SITES[col]),'UNIV']='ok'
    df_UUCR_SITES.loc[df_UUCR_SIEGE[col].isin(df_UUCR_SITES[col]),'TYPE_SITE']='site_princ'
    df_UUCR_SITES.loc[df_UUCR_UNIVERSITAIRE[col].isin(df_UUCR_SITES[col]),'TYPE_SITE']='site_sec'
    df_UUCR_SITES.loc[(-df_UUCR_UNIVERSITAIRE[col].isin(df_UUCR_SITES[col]))|(-df_UUCR_SIEGE.isin(df_UUCR_SITES)[col]),'TYPE_SITE']='site_sec'
    
    return df, df_COM_INV, df_COM_SITES, df_UUCR_INV, df_UUCR_SITES


In [ ]:
def gentab3(df):
    df1=df.loc[df['RGP3'] not in ['CPGE','STS','UNIV','GE','INP','UT','ESPE'],:]
    array=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','RGP3']
    f = {'EFFTOT': 'sum'}
    df_COM2_AUT=df1.groupby(array, as_index=False).agg(f)
    
    df1=df.loc[df['RGP3'] in ['CPGE','STS'],:]
    array=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','RGP3']
    f = {'EFFTOT': 'sum'}
    df_COM2_CSI=df1.groupby(array, as_index=False).agg(f)

    df1=df.loc[df['RGP3'] in ['UNIV','GE','INP','UT'] & df['IUT']!='IUT' & df['INSPE']!='INSPE' & df['RGP2']!='EPE',:]
    array=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','RGP3','RGP2','COM_M']
    f = {'EFFTOT': 'sum'}
    df_COM2_UNIV=df1.groupby(array, as_index=False).agg(f)

    df1=df.loc[df['IUT'] == 'IUT',:]
    array=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','RGP3','RGP2','COM_M']
    f = {'EFFTOT': 'sum'}
    df_COM2_IUT=df1.groupby(array, as_index=False).agg(f)
    df_COM2_IUT.loc[:,'RGP3']="IUT"

    df1=df.loc[df['INSPE'] == 'ESPE',:]
    array=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','INSPE','COM_M']
    f = {'EFFTOT': 'sum'}
    df_COM2_ESPE=df1.groupby(array, as_index=False).agg(f)
    df_COM2_ESPE.loc[:,'RGP3']="INSPE"

    df1=df.loc[df['RGP2'] == 'EPE',:]
    array=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1','RGP2','COM_M']
    f = {'EFFTOT': 'sum'}
    df_COM2_EPE=df1.groupby(array, as_index=False).agg(f)
    df_COM2_EPE['RGP3']=df1.loc[:,'RGP2']

    df_COM2_UNIV.loc[df['COM_U']!=1,'RGP3']=f"{df['RGP3']}_S"
    (df_COM2_CSI.pivot_table(index=['RENTREE','REG_ID','UUCR_ID','DEP_ID','COM_CODE1','COM_NOM1'], columns=['RGP3','EFFTOT'], values='spend')
              .add_prefix('spend_')
              .reset_index())
    
    df_COM2_CSI.loc[df_COM2_CSI['CPGE']!='.'&df_COM2_CSI['STS']=='.','RGP3']='CPGE'
    df_COM2_CSI.loc[df_COM2_CSI['CPGE']!='.'&df_COM2_CSI['STS']!='.','RGP3']='STS_CPGE'
    df_COM2_CSI.loc[df_COM2_CSI['CPGE']=='.'&df_COM2_CSI['STS']!='.','RGP3']='STS'
    df_COM2_CSI['EFFTOT']=sum(df_COM2_CSI['CPGE'],df_COM2_CSI['STS'])
    del df_COM2_CSI['CPGE']
    del df_COM2_CSI['STS']

    df_COM2_INV=pd.concat([df_COM2_UNIV,df_COM2_CSI,df_COM2_AUT,df_COM2_IUT,df_COM2_ESPE])
    df_COM2_INV.loc[:,'UAI']=f"{df.loc[:,'COMECODE1']}_{df.loc[:,'RGP3']}"
    df_COM2_INV.loc[:,'GEO_LIB']=df.loc[:,'COM_NOM1']
    df_COM2_INV.loc[:,'GEO_ID']=df.loc[:,'COME_CODE1']
    df_COM2_INV.loc[:,'REF_ID']=df.loc[:,'DEP_ID']
    df_COM2_INV.loc[:,'TYPE2']=df.loc[:,'RGP3']
    del df_COM2_INV['RENTREE']
    del df_COM2_INV['DEP_ID']
    del df_COM2_INV['COM_CODE1']
    del df_COM2_INV['COM_NOM1']
    del df_COM2_INV['RGP3']
    del df_COM2_INV['RGP2']
    del df_COM2_INV['INSPE']

    df_COM2_SIEGE_UNIV=df.loc[df['COM_U']==1 & df['RGP3'] == 'UNIV',:]
    df_COM2_SIEGE_UNIV.loc[:,'GEO_LIB']=df.loc[:,'COM_NOM1']
    df_COM2_SIEGE_UNIV.loc[:,'GEO_ID']=df.loc[:,'COME_CODE1']
    df_COM2_SIEGE_UNIV.loc[:,'REF_ID']=df.loc[:,'DEP_ID']

    array=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    f = {'EFFTOT': 'sum'}
    df_COM2_SIEGE_UNIV=df_COM2_SIEGE_UNIV.groupby(array, as_index=False).agg(f)

    df_COM2_SIEGE=df.loc[df['COM_U']==1 & (df['RGP3'] in ['UNIV','GE','INP','UT','ENS','ESPE','IUFM']| df['RGP_ING']== 'ING_MESR_PU'),:]
    df_COM2_SIEGE.loc[:,'GEO_LIB']=df.loc[:,'COM_NOM1']
    df_COM2_SIEGE.loc[:,'GEO_ID']=df.loc[:,'COME_CODE1']
    df_COM2_SIEGE.loc[:,'REF_ID']=df.loc[:,'DEP_ID']

    array=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    f = {'EFFTOT': 'sum'}
    df_COM2_SIEGE=df_COM2_SIEGE.groupby(array, as_index=False).agg(f)

    df_COM2_UNIV=df.loc[df['RGP3'] =='UNIV',:]
    df_COM2_UNIV.loc[:,'GEO_LIB']=df.loc[:,'COM_NOM1']
    df_COM2_UNIV.loc[:,'GEO_ID']=df.loc[:,'COME_CODE1']
    df_COM2_UNIV.loc[:,'REF_ID']=df.loc[:,'DEP_ID']

    array=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    f = {'EFFTOT': 'sum'}
    df_COM2_UNIV=df_COM2_UNIV.groupby(array, as_index=False).agg(f)

    df_COM2_UNIVERSITAIRE=df.loc[df['RGP3'] in ['UNIV','GE','INP','UT','ENS','ESPE','IUFM']| df['RGP_ING']== 'ING_MESR_PU',:]
    df_COM2_UNIVERSITAIRE.loc[:,'GEO_LIB']=df.loc[:,'COM_NOM1']
    df_COM2_UNIVERSITAIRE.loc[:,'GEO_ID']=df.loc[:,'COME_CODE1']
    df_COM2_UNIVERSITAIRE.loc[:,'REF_ID']=df.loc[:,'DEP_ID']

    array=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    f = {'EFFTOT': 'sum'}
    df_COM2_UNIVERSITAIRE=df_COM2_UNIVERSITAIRE.groupby(array, as_index=False).agg(f)
    
    df_COM2_SITES=df_COM2_INV.groupby(array, as_index=False).agg(f)

    col=['REG_ID','UUCR_ID','REF_ID','GEO_ID','GEO_LIB']
    df_COM2_SITES.loc[df_COM2_UNIVERSITAIRE[col].isin(df_COM2_SITES[col]),'UNIVERSITAIRE']='ok'
    df_COM2_SITES.loc[df_COM2_SIEGE[col].isin(df_COM2_SITES[col]),'SIEGE']='ok'
    df_COM2_SITES.loc[df_COM2_SIEGE_UNIV[col].isin(df_COM2_SITES[col]),'SIEGE_UNIV']='ok'
    df_COM2_SITES.loc[df_COM2_UNIV[col].isin(df_COM2_SITES[col]),'UNIV']='ok'
    df_COM2_SITES.loc[df_COM2_SIEGE[col].isin(df_COM2_SITES[col]),'TYPE_SITE']='site_princ'
    df_COM2_SITES.loc[df_COM2_UNIVERSITAIRE[col].isin(df_COM2_SITES[col]),'TYPE_SITE']='site_sec'
    df_COM2_SITES.loc[(-df_COM2_UNIVERSITAIRE[col].isin(df_COM2_SITES[col]))|(-df_COM2_SIEGE.isin(df_COM2_SITES)[col]),'TYPE_SITE']='site_sec'
    
    return df_COM2_INV, df_COM2_SITES

In [ ]:
def gentab4(df_COM_SITES, df_COM2_SITES, df_UUCR_SITES):
    
    df_COM_SITES=df_COM_SITES[['GEO_ID','TYPE_SITE']]
    df_COM2_SITES=df_COM2_SITES[['GEO_ID','TYPE_SITE']]
    df_UUCR_SITES=df_UUCR_SITES[['GEO_ID','TYPE_SITE']]
    df=pd.concat([df_COM_SITES,df_COM2_SITES,df_UUCR_SITES])
    df.loc[:,'un']=1

    array=['GEO_ID','TYPE_SITE']
    f = {'un': 'sum'}
    df_les_sites=df.groupby(array, as_index=False).agg(f)
    
    return df_les_sites

In [ ]:
def eff_sans_double_compte21(df):
    df1=df[(np.isna(df['OPPOS']) | df['OPPOS']=='O')| df['SECT']!='PU' ]
    array=['ETABLI','SECT','OPPOS']
    f = {'EFFECTIF_SANS_DOUBLE_COMPTE': 'sum'}
    df_OPPOS=df1.groupby(array, as_index=False).agg(f)

    array=['ETABLI','SECT','OPPOS']
    f = {'EFFECTIF_SANS_DOUBLE_COMPTE': 'sum'}
    df_OPPOS=df.groupby(array, as_index=False).agg(f)

    return df_OPPOS,df_format